In [ ]:
# -*- coding: utf-8 -*-
#-*- coding: cp950 -*-

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
from typing import *
import torch
import torch.optim as optim
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os

from google.colab import drive
drive.mount('/content/gdrive',force_remount = True)
base_dir = '/content/gdrive/MyDrive'
path = Path(base_dir +'/Keras_tutorial')  #imgs
# path.mkdir(parents=True,exist_ok=True)
os.chdir(path)

Mounted at /content/gdrive


In [ ]:
#Pre-Trained Model as Classifier
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16

# load an image from file
image = load_img('./images/elephant.jpg', target_size=(224, 224))

# convert the image pixels to a numpy array
image = img_to_array(image)

# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

# prepare the image for the VGG model
image = preprocess_input(image)

# load the model
model = VGG16()

# predict the probability across all output classes
yhat = model.predict(image)

# convert the probabilities to class labels
label = decode_predictions(yhat)

# retrieve the most likely result, e.g. highest probability
label = label[0][0]

# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))

In [ ]:
>>>Pre-Trained Model as Feature Extractor Preprocessor
The pre-trained model may be used as a standalone program to extract features from new photographs.

Specifically, the extracted features of a photograph may be a vector of numbers 
that the model will use to describe the specific features in a photograph. 
These features can then be used as input in the development of a new model.

The last few layers of the VGG16 model are fully connected layers prior to the output layer.
These layers will provide a complex set of features to describe a given input image 
and may provide useful input when training a new model for image classification or related computer vision task.

The image can be loaded and prepared for the model, as we did before in the previous example.
We will load the model with the classifier output part of the model, 
but manually remove the final output layer. 
This means that the second last fully connected layer with 4,096 nodes will be the new output layer.

In [ ]:
# example of using the vgg16 model as a feature extraction model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from keras.models import Model
from pickle import dump

# load an image from file
image = load_img('./images/elephant.jpg', target_size=(224, 224))

# convert the image pixels to a numpy array
image = img_to_array(image)

# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

# prepare the image for the VGG model
image = preprocess_input(image)

# load model
model = VGG16()

# remove the output layer
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

# get extracted features
features = model.predict(image)
print(features.shape)

# save to file
dump(features, open('./images/elephant.pkl', 'wb'))

In [ ]:
Running the example loads the photograph, then prepares the model as a feature extraction model.

The features are extracted from the loaded photo and the shape of the feature vector is printed, 
showing it has 4,096 numbers. This feature is then saved to a new file dog.pkl in the current working directory.

In [ ]:
#########################################
>>>Pre-Trained Model as Feature Extractor in Model
We can use some or all of the layers in a pre-trained model as a feature extraction component of a new model directly.
This can be achieved by loading the model, 
then simply adding new layers. This may involve adding new convolutional and pooling layers to expand upon the feature extraction capabilities of the model or adding new fully connected classifier type layers to learn how to interpret the extracted features on a new dataset, or some combination.
For example, we can load the VGG16 models without the classifier part of the model 
by specifying the “include_top” argument to “False”, 
and specify the preferred shape of the images in our new dataset as 300×300.

In [ ]:
We can then use the Keras function API to add a new Flatten layer after the last pooling layer in the VGG16 model,
then define a new classifier model with a Dense fully connected layer 
and an output layer that will predict the probability for 10 classes

In [ ]:
An alternative approach to adding a Flatten layer would be to define the VGG16 model with an average pooling layer,
and then add fully connected layers. 
Perhaps try both approaches on your application and see which results in the best performance.

The weights of the VGG16 model and the weights for the new model will all be trained together on the new dataset.

In [ ]:
# tending the vgg16 model
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
# load model without classifier layers
model = VGG16(include_top=False, input_shape=(300, 300, 3))

# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(10, activation='softmax')(class1)

# define new model
model = Model(inputs=model.inputs, outputs=output)

# summarize
model.summary()

In [ ]:
We can see that we have flattened the output of the last pooling layer and added our new fully connected layers.

In [ ]:
Alternately, we may wish to use the VGG16 model layers, 
but train the new layers of the model without updating the weights of the VGG16 layers. 
This will allow the new output layers to learn to interpret the learned features of the VGG16 model.

This can be achieved by setting the “trainable” property on each of the layers in the loaded VGG model to False prior to training.
For example:

In [ ]:
# load model without classifier layers
model = VGG16(include_top=False, input_shape=(300, 300, 3))

# mark loaded layers as not trainable
for layer in model.layers:
layer.trainable = False

In [ ]:
You can pick and choose which layers are trainable.

In [ ]:
For example, perhaps you want to retrain some of the convolutional layers deep in the model, 
but none of the layers earlier in the model. For example:

In [ ]:
# load model without classifier layers
model = VGG16(include_top=False, input_shape=(300, 300, 3))
# mark some layers as not trainable
model.get_layer('block1_conv1').trainable = False
model.get_layer('block1_conv2').trainable = False
model.get_layer('block2_conv1').trainable = False
model.get_layer('block2_conv2').trainable = False